In [1]:
import numpy as np
import pandas as pd

######  Загружаю данные

In [2]:
# загружаю  данные 

#traidisplay pd.read_csv('DATA_data_fusion_contest_2022/train.csv')
#pd.read_csv("../input/vtbdatafusion2022/train.csv")
#print('train.csv - DONE!')


#tra_val = pd.read_csv('DATA_data_fusion_contest_2022/train_matching.csv')
#pd.read_csv('../input/vtbdatafusion2022/train_matching.csv')
#print('train_matching.csv - DONE!')


#click_data = pd.read_csv('DATA_data_fusion_contest_2022/clickstream.csv') #очень аккуратно надо выполнять!! 
#pd.read_csv('../input/vtbdatafusion2022/clickstream.csv')
#print('clickstream.csv - DONE!')


trans = pd.read_csv('DATA_data_fusion_contest_2022/transactions.csv')
#trans = pd.read_csv('../input/vtbdatafusion2022/transactions.csv')
print('transactions.csv - DONE!')


#sub_data = pd.read_csv('DATA_data_fusion_contest_2022/sample_submission.csv')
#sub_data = pd.read_csv('../input/vtbdatafusion2022/sample_submission.csv')
#print('sample_submission.csv - DONE!')


df_clik_h_edu_no = pd.read_csv('newdatadatafusion/df_clik_h_edu_no2.csv')
#df_clik_h_edu_no = pd.read_csv('../input/df2022-converted/df_clik_h_edu_no.csv')
print('df_clik_h_edu_no - DONE!')


df_clik_h_edu_only = pd.read_csv('newdatadatafusion/df_clik_h_edu_only2.csv')
#df_clik_h_edu_only = pd.read_csv('../input/df2022-converted/df_clik_h_edu_only.csv')
print('df_clik_h_edu_only - DONE!')


transactions.csv - DONE!
df_clik_h_edu_no - DONE!
df_clik_h_edu_only - DONE!


######  разделяю на 2 разные таблицы с выщкой и без неё


In [4]:
display(df_clik_h_edu_no)
display(df_clik_h_edu_only)

In [ ]:
train_h_edu_only = train[train.higher_education==1]
#train_h_edu_only.head

train_h_edu_no = train[train.higher_education==0]
#train_h_edu_no.head

###### вышенайденным людям нахожу их rkt -> после надо взять самый большой файл и скрестить их. а так же переименую чтобы потом скрестить таблицы

In [ ]:
df_h_edu_only = train_h_edu_only.merge(tra_val)
df_h_edu_no = train_h_edu_no.merge(tra_val)


df_h_edu_no.rename(columns = {'rtk' : 'user_id'}, inplace = True)
df_h_edu_only.rename(columns = {'rtk' : 'user_id'}, inplace = True)

######  опять скрещиваю таблицы 

In [ ]:
#таблички идут на свидание
df_clik_h_edu_no = click_data.merge(df_h_edu_no)

df_clik_h_edu_only = click_data.merge(df_h_edu_only)

###### а так же сохраню в в отдельный файл

In [ ]:
#df_clik_h_edu_no.to_csv('df_clik_h_edu_no.csv', sep='\t', encoding='utf-8')
#df_clik_h_edu_only.to_csv('df_clik_h_edu_only.csv', sep='\t', encoding='utf-8')
#к имени надо добавлять .csv

In [ ]:
#посмотрю размер
print(df_clik_h_edu_no.shape , df_clik_h_edu_only.shape)

In [ ]:
#переименовать графу времени!
#trans.rename (columns = {'transaction_dttm': 'timestamp'}, inplace = True)
#display(trans)

In [ ]:
df_all_h_edu_no = trans.merge(df_clik_h_edu_no)
#df_all_h_edu_no # тут пока что лажа. чуть позже еще подумаю
# все фигня. по дате нет совпадений.

#  работа с данными которые надо предсказать


In [4]:
sub_data

In [5]:
sub_data.rename(columns={'bank':'user_id'}, inplace = True)
sub_data

In [ ]:
# таким образом я узнал, что можно обращаться по индексам
#sub_data.bank[0]

In [ ]:
#my_df["range"].isin([45]).any()
#trans.user_id.isin([sub_data.bank[0]]).any()

In [ ]:
# появиль новые интересные факты...
#щя будем смотреть

In [6]:
df_sub_data = sub_data.merge(trans)
df_sub_data


In [15]:
#так я узнал для всех ли нашлись траты

n = len(pd.unique(df_sub_data['user_id'])) 
n 

In [7]:
# убрал mcc = -1 т.к. такого кода нет
df_sub_data = df_sub_data.loc[df_sub_data['mcc_code'] != -1]
df_sub_data

In [14]:
#df_sub_data = df_sub_data.reset_index()
js = np.array(df_sub_data.index)
js = js[1:]
js = max(js)
js

### без учета валюты 

In [32]:
mon = np.zeros( len(pd.unique(df_sub_data['user_id'])) )
col_VO = np.zeros( len(pd.unique(df_sub_data['user_id'])) )
i = 0
j = 0
#cou = 0
#df_sub_data.reset_index()
#js = np.array(df_sub_data.index)
#js = js[1:]

mon[i] += float(df_sub_data.transaction_amt[j]) 
# Н учесть первый эл-т тк далее я добав сразу j+1

for j in  range (js-1):
    #print(j)
    if j/(js//100) == int(j/(js//100)):
        print(j/(js//100),'%')
        
    
    h1 = str( df_sub_data.user_id[j] )
    h2 = str( df_sub_data.user_id[j + 1] )
    
    if h1 == h2: #проверяю нынешний и след
        #cou += 1
        
        m2 = float(df_sub_data.transaction_amt[j+1])
        
        col_VO[i] += 1
        mon[i] += m2
    else:
        
        # типо до этого я плюсовал только j+1 так что каджый раз проспускал первый эл-т
        mon [i+1] += float(df_sub_data.transaction_amt[j+1]) 
        
        i += 1
print('mon\n',mon)
print('col-VO\n',col_VO)
cr_tran = mon[:]/col_VO[:]
print('cr_tran')
print(cr_tran)


In [45]:
#добавлю новые столб and save file
#sub_data['all_mon'] = mon
#sub_data['col_VO_tran'] = col_VO
#sub_data['cr_tran'] = cr_tran
#sub_data./

sub_data.to_csv('./sub_data_new.csv', index = False)#, sep='\t', encoding='utf-8')

In [34]:
n = len(pd.unique(df_sub_data['currency_rk'])) 
print(n)
df_sub_data

In [ ]:
print(mon)

In [ ]:
he = float(df_sub_data.transaction_amt[j])
type(he)

In [ ]:
 if df_sub_data.user_id[0] == sub_data.user_id[0]:
        print(float(df_sub_data.transaction_amt[0]))

In [ ]:
sub_data

In [ ]:
df_sub_data = df_sub_data.reset_index()
df_sub_data

In [52]:
 #del #sub_data
display(df_clik_h_edu_no)
df_clik_h_edu_no.user_id
#trans.rename(columns={})

In [9]:
trans.shape[0]

19821910

In [12]:

size = trans.shape[0]
mon2 = np.zeros( len(pd.unique(trans['user_id']))  ) 
col_VO2 = np.zeros( len(pd.unique(trans['user_id'])) )
i = 0
j = 0

for j in  range(size-1):
    
    if j/(size//100) == int(j/(size//100)):
        print(j/(size//100),'%')
        
    u1 = str(trans.user_id[i])
    u2 = str(trans.user_id[i+1])
    
    if u1 == u2:
        m2 = float( trans.transaction_amt[j+1] )
        col_VO2[i] += 1
        mon2 +=m2
    else:
        mon2[i+1] += float( trans.transaction_amt[j+1])
        i += 1
        
        
print('mon\n',mon2)
print('col-VO\n',col_VO2)
cr_tran = mon2[:]/col_VO2[:]
print('cr_tran')
print(cr_tran)

0.0 %
1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %


KeyboardInterrupt: 

In [13]:
#df_clik_h_edu_no.pop('Unnamed: 0')
#df_clik_h_edu_no.rename(columns={})

,user_id,cat_id,timestamp,new_uid,bank,higher_education
0,003a54bc170f4e109036fe534f78c7af,535,2021-01-27 10:07:46,931116,6a6ac0a2e96f4ef98829c0d720b62f09,0
1,003a54bc170f4e109036fe534f78c7af,25,2021-02-03 07:22:16,931116,6a6ac0a2e96f4ef98829c0d720b62f09,0
2,003a54bc170f4e109036fe534f78c7af,165,2021-02-04 02:02:34,931116,6a6ac0a2e96f4ef98829c0d720b62f09,0
3,003a54bc170f4e109036fe534f78c7af,931,2021-02-06 01:31:54,931116,6a6ac0a2e96f4ef98829c0d720b62f09,0
4,003a54bc170f4e109036fe534f78c7af,503,2021-02-06 14:07:00,931116,6a6ac0a2e96f4ef98829c0d720b62f09,0
...,...,...,...,...,...,...
1048570,1f627a283e9d4f97a28c1bec5c224906,165,2021-07-03 16:34:00,1774470,118bb16b0371465eba8c2d3784a62ca6,0
1048571,1f627a283e9d4f97a28c1bec5c224906,165,2021-07-03 17:24:00,1774470,118bb16b0371465eba8c2d3784a62ca6,0
1048572,1f627a283e9d4f97a28c1bec5c224906,165,2021-07-03 21:15:00,1774470,118bb16b0371465eba8c2d3784a62ca6,0
1048573,1f627a283e9d4f97a28c1bec5c224906,165,2021-07-03 22:32:00,1774470,118bb16b0371465eba8c2d3784a62ca6,0


In [18]:
trans.rename(columns={'user_id': 'bank'}, inplace = True)
trans
#df_tenm

,bank,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,000932580e404dafbecd5916d4640938,5411,48,-361.072300,2020-08-03 08:05:23
1,000932580e404dafbecd5916d4640938,5499,48,-137.313980,2020-08-05 01:27:40
2,000932580e404dafbecd5916d4640938,5499,48,-138.849810,2020-08-05 03:28:11
3,000932580e404dafbecd5916d4640938,4829,48,-309.476530,2020-08-06 00:36:29
4,000932580e404dafbecd5916d4640938,5411,48,-133.473700,2020-08-09 00:30:13
...,...,...,...,...,...
19821905,fffb051877b949f49bed6538393a06ae,5411,48,-40.727768,2021-07-31 05:33:03
19821906,fffb051877b949f49bed6538393a06ae,5411,48,-277.494350,2021-07-31 08:57:02
19821907,fffb051877b949f49bed6538393a06ae,5411,48,-138.097350,2021-07-31 08:59:33
19821908,fffb051877b949f49bed6538393a06ae,6011,48,-86.046776,2021-08-01 23:04:41


In [ ]:
df_tenp = trans.merge(df_clik_h_edu_no)
df_temp